In [2]:
import pandas as pd
import re
import datetime

In [3]:
df = pd.read_excel(r'C:\Users\leonb\.venv\Shark Attack\GSAF5.xls', engine="xlrd")

In [4]:
df.columns

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal Y/N', 'Time', 'Species ',
       'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22'],
      dtype='object')

In [ ]:
# Goal: Commit the perfect shark murder
# Problem Statement: Identify the correct location, activity,  time of the day and what time of the year

# Hypothesis 1: The perfect location is Australia (can be narrowed down on state level if fitting to the country)
# Hypothesis 2: The perfect time of the year is early in the year on a morning
# Hypothesis 3: The deadliest sharks will be white, tiger and bull

# Columns we need: "Fatal Y/N", "Country", "Date", "Activity",  "Time"
# -> GFM: Fatal & Country
# -> LB: Date
# -> MB: Activty & Time


# further stuf:
# - age, gender of the person we try to kill
# species for "e.g should be killed by white shark"





In [5]:
pd.set_option('display.max_rows', None)
df['Date']

0                                     2025-02-22 00:00:00
1                                     2025-02-22 00:00:00
2                                     2025-02-22 00:00:00
3                                     2025-02-10 00:00:00
4                                     2025-02-10 00:00:00
5                                     2025-02-07 00:00:00
6                                     2025-02-03 00:00:00
7                                     2025-01-23 00:00:00
8                                     2025-01-22 00:00:00
9                                     2025-01-03 00:00:00
10                                    2025-01-11 00:00:00
11                                    2025-01-02 00:00:00
12                                    2025-01-02 00:00:00
13                                    2024-12-30 00:00:00
14                                    2024-12-29 00:00:00
15                                    2024-12-29 00:00:00
16                                    2024-12-28 00:00:00
17            

In [6]:
df["Date"] = df["Date"].astype(str).str.replace(r"\s*00:00:00", "", regex=True)
# Define a regex pattern for valid dates
valid_date_pattern = r"^\d{2}-[A-Za-z]{3}-\d{4}$|^[A-Za-z]{3}-\d{4}$"

# Create a mask to filter correct dates
correct_dates = df[df["Date"].str.match(valid_date_pattern, na=False)]

# Store incorrect dates separately for cleaning
incorrect_dates = df[~df["Date"].str.match(valid_date_pattern, na=False)]
# Define unwanted keywords
unwanted_patterns = r"(no date|world war|before|after|between|c[.\s]?a[.\s]?|circa|approx|around|B\.?C\.?|BCE|A\.?D\.?|CE|prior|\bOR\b)"

# Drop rows containing any of these words
incorrect_dates = incorrect_dates[~incorrect_dates["Date"].apply(lambda x: bool(re.search(unwanted_patterns, str(x), re.IGNORECASE)))]
# Remove the words but keep the rest of the date
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(r"\b(Reported|Late|Early)\b", "", case=False, regex=True).str.strip()
# Define regex pattern for standalone years or decades
standalone_year_pattern = r"^\d{4}s?$"  

# Drop rows that match exactly a 4-digit year or decade
incorrect_dates = incorrect_dates[~incorrect_dates["Date"].str.match(standalone_year_pattern, na=False)]

# Fix missing hyphens in "DD MMM YYYY" and "DD-MMM YYYY" formats
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(
    r"(\d{1,2})\s([A-Za-z]+)-?\s?(\d{4})",  # Matches '22 Jul-2023', '5 Jul 2014', '3-Jul 2023'
    r"\1-\2-\3",  # Converts to '22-Jul-2023', '5-Jul-2014'
    regex=True
)
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(
    r"(\d{4})\.(\d{2})\.(\d{2})", r"\3-\2-\1", regex=True  # Rearranges YYYY.MM.DD → DD-MM-YYYY
)
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(
    r"(\d{4}\.\d{2}\.\d{2})\.\w$", r"\1", regex=True  # Removes the trailing single letter
)
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(
    r"^[\.\s]+", "", regex=True  # Removes leading dots and spaces
)
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(
    r"(\d{1,2})-(Decp)(\d{4})", r"\1-Dec-\3", regex=True
)
# Fix missing hyphens in "DD-MMM YYYY" (e.g., "08-Jun 2023" → "08-Jun-2023")
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(
    r"(\d{1,2}-[A-Za-z]{3}) (\d{4})", r"\1-\2", regex=True
)

# Fix missing hyphens in "DD MMM-YYYY" (e.g., "13-May 1871" → "13-May-1871")
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(
    r"(\d{1,2}) ([A-Za-z]{3}-\d{4})", r"\1-\2", regex=True
)
# 1️⃣ Convert YYYY-MM-DD → DD-MMM-YYYY  (e.g., "2025-02-22" → "22-Feb-2025")
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(
    r"(\d{4})-(\d{2})-(\d{2})", r"\3-\2-\1", regex=True
)

# 2️⃣ Fix ordinal days (1st, 2nd, 3rd, etc.) → Normal numbers  (e.g., "3rd-Oct-2016" → "03-Oct-2016")
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(
    r"(\d{1,2})(st|nd|rd|th)-", r"\1-", regex=True
)

# 3️⃣ Standardize month names (e.g., "31-July-2009" → "31-Jul-2009")
month_fix = {
    "January": "Jan", "February": "Feb", "March": "Mar", "April": "Apr", "May": "May", "June": "Jun",
    "July": "Jul", "August": "Aug", "September": "Sep", "October": "Oct", "November": "Nov", "December": "Dec"
}
for long_month, short_month in month_fix.items():
    incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(long_month, short_month, regex=True)

# 4️⃣ Add leading zero to single-digit days (e.g., "9-Mar-2018" → "09-Mar-2018")
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(
    r"\b(\d{1})-([A-Za-z]{3}-\d{4})", r"0\1-\2", regex=True
)

# 5️⃣ Fix incorrect "Month-Day-Year" order → Correct "Day-Month-Year" (e.g., "May-17-1803" → "17-May-1803")
incorrect_dates["Date"] = incorrect_dates["Date"].str.replace(
    r"([A-Za-z]{3})-(\d{1,2})-(\d{4})", r"\2-\1-\3", regex=True
)
def convert_numeric_to_text_date(date_str):
    """ Convert DD-MM-YYYY → DD-MMM-YYYY """
    try:
        return datetime.datetime.strptime(date_str, "%d-%m-%Y").strftime("%d-%b-%Y")
    except ValueError:
        return date_str  # If it can't be converted, return original

# Apply the function to the Date column
incorrect_dates["Date"] = incorrect_dates["Date"].apply(convert_numeric_to_text_date)
# Reuse the same regex pattern to filter valid dates
new_correct_dates = incorrect_dates[incorrect_dates["Date"].str.match(valid_date_pattern, na=False)]

# Store newly cleaned correct dates
correct_dates = pd.concat([correct_dates, new_correct_dates])

# Keep only still-incorrect dates for further cleaning
incorrect_dates = incorrect_dates[~incorrect_dates["Date"].str.match(valid_date_pattern, na=False)]
# Convert 'Date' column to datetime for sorting, ignore errors to keep text-based dates
correct_dates["Date_Sort"] = pd.to_datetime(correct_dates["Date"], format="%d-%b-%Y", errors="coerce")

# Sort by 'Date_Sort' while keeping original 'Date' column intact
correct_dates = correct_dates.sort_values(by="Date_Sort").drop(columns=["Date_Sort"])

# Reset index
correct_dates = correct_dates.reset_index(drop=True)

# Extract year, month, and day separately for sorting
correct_dates["Year"] = correct_dates["Date"].str.extract(r"(\d{4})").astype(float)
correct_dates["Month"] = correct_dates["Date"].str.extract(r"([A-Za-z]{3})")
correct_dates["Day"] = correct_dates["Date"].str.extract(r"(^\d{1,2})").astype(float)

# Convert month names to numbers for proper sorting
month_map = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, 
             "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}
correct_dates["Month"] = correct_dates["Month"].map(month_map)

# Fill NaNs with extreme values to ensure correct sorting
correct_dates["Day"] = correct_dates["Day"].fillna(0)  # No day → 0 (comes last within same month)
correct_dates["Month"] = correct_dates["Month"].fillna(0)  # No month → 0 (comes last within same year)

# Sort in descending order: first by Year, then Month, then Day
correct_dates = correct_dates.sort_values(by=["Year", "Month", "Day"], ascending=[False, False, False])

# Drop helper columns
correct_dates = correct_dates.drop(columns=["Year", "Month", "Day"]).reset_index(drop=True)

In [7]:
len(correct_dates)

6367